In [ ]:
# Load the extension that allows us to compile CUDA code in python notebooks
# Documentation is here: https://nvcc4jupyter.readthedocs.io/en/latest/
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc4jupyter



^C
Source files will be saved in "C:\Users\A1742\AppData\Local\Temp\tmp760j_1bm".


  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to c:\users\a1742\appdata\local\temp\pip-req-build-haocike4
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 28f872a2f99a1b201bcd0db14fdbc5a496b9bfd7
  Installing build dependencies: started
  Installing build dependencies: still running...
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git 'C:\Users\A1742\AppData\Local\Temp\pip-req-build-haocike4'

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\A1742\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)


In [ ]:
%%cuda_group_save -g "knn2" -n "main.cu"

// Required header files
#include <iostream>
#include <fstream>
#include <vector>
#include <string>
#include <cstring>
#include <algorithm>
#include <cuda_runtime.h>
#include <cfloat>
#include <chrono>    // For timing execution

// Constants definition
#define THREADS 256        // Number of threads per block
#define IMAGESIZE 784      // Image size (28x28 = 784 pixels)

// Function to handle big-endian to little-endian conversion
uint32_t swap32(uint32_t val) {
    val = ((val << 8) & 0xFF00FF00) | ((val >> 8) & 0xFF00FF);
    return (val << 16) | (val >> 16);
}

// structure to store training/testing samples
struct TrainingSample {
    int label;                  // The digit (0-9)
    float image[IMAGESIZE];     // Normalized pixel values
};

__global__ void bitonicSortStep(float* d_distances, int* d_labels, int j, int k, int num_samples) {
    unsigned int i = threadIdx.x + blockDim.x * blockIdx.x;
    if (i >= num_samples) return;

    unsigned int ixj = i ^ j;

    if (ixj > i && ixj < num_samples) {
        // determine the sorting direction
        if ((i & k) == 0) {
            // sort in ascending order
            if (d_distances[i] > d_distances[ixj]) {
                // swap distances
                float temp_dist = d_distances[i];
                d_distances[i] = d_distances[ixj];
                d_distances[ixj] = temp_dist;

                // swap corresponding labels
                int temp_label = d_labels[i];
                d_labels[i] = d_labels[ixj];
                d_labels[ixj] = temp_label;
            }
        } else {
            // sort in descending order
            if (d_distances[i] < d_distances[ixj]) {
                // swap distances
                float temp_dist = d_distances[i];
                d_distances[i] = d_distances[ixj];
                d_distances[ixj] = temp_dist;

                // swap corresponding labels
                int temp_label = d_labels[i];
                d_labels[i] = d_labels[ixj];
                d_labels[ixj] = temp_label;
            }
        }
    }
}


void bitonicSort(float* d_distances, int* d_labels, int num_samples) {
    // Calculate the next power of two
    int pow2_size = 1;
    while (pow2_size < num_samples) pow2_size <<= 1;

    // Pad the distances and labels with maximum values
    int padded_size = pow2_size;
    if (padded_size > num_samples) {
        float max_distance = FLT_MAX;
        int max_label = -1; // Use an invalid label for padding

        // Create temporary arrays for padding
        float* h_pad_distances = new float[padded_size - num_samples];
        int* h_pad_labels = new int[padded_size - num_samples];
        for (int i = 0; i < padded_size - num_samples; ++i) {
            h_pad_distances[i] = max_distance;
            h_pad_labels[i] = max_label;
        }

        // Copy padding data to device
        cudaMemcpy(d_distances + num_samples, h_pad_distances, (padded_size - num_samples) * sizeof(float), cudaMemcpyHostToDevice);
        cudaMemcpy(d_labels + num_samples, h_pad_labels, (padded_size - num_samples) * sizeof(int), cudaMemcpyHostToDevice);

        // Free temporary host arrays
        delete[] h_pad_distances;
        delete[] h_pad_labels;
    }

    // Set up grid and block dimensions
    dim3 block(THREADS);
    dim3 grid((padded_size + block.x - 1) / block.x);

    // Main sorting loops
    for (int k = 2; k <= pow2_size; k <<= 1) {
        for (int j = k >> 1; j > 0; j >>= 1) {
            bitonicSortStep<<<grid, block>>>(d_distances, d_labels, j, k, padded_size);
            cudaDeviceSynchronize();
        }
    }
}


// CUDA kernel for computing Euclidean distances
__global__ void computeEuclideanDistances(float* d_images, float* d_testImage,
                                        float* d_distances, int* d_labels,
                                        int* d_train_labels, int num_samples) {
    // calculate global thread ID
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < num_samples) {
        float sum = 0.0f;
        // calculate squared euclidean distance
        for (int i = 0; i < IMAGESIZE; ++i) {
            float diff = d_images[idx * IMAGESIZE + i] - d_testImage[i];
            sum += diff * diff;
        }
        d_distances[idx] = sqrtf(sum);  // Store the distance
        d_labels[idx] = d_train_labels[idx];  // Store the label
    }
}

// function to load MNIST dataset in IDX format
bool loadMNISTImages(const std::string& image_path, const std::string& label_path,
                    std::vector<TrainingSample>& samples) {
    // Open image file
    std::ifstream image_file(image_path, std::ios::binary);
    if (!image_file) {
        std::cerr << "Cannot open image file: " << image_path << std::endl;
        return false;
    }

    // Open label file
    std::ifstream label_file(label_path, std::ios::binary);
    if (!label_file) {
        std::cerr << "Cannot open label file: " << label_path << std::endl;
        return false;
    }

    // Read image file header
    uint32_t magic, num_items, num_rows, num_cols;
    image_file.read(reinterpret_cast<char*>(&magic), sizeof(magic));
    image_file.read(reinterpret_cast<char*>(&num_items), sizeof(num_items));
    image_file.read(reinterpret_cast<char*>(&num_rows), sizeof(num_rows));
    image_file.read(reinterpret_cast<char*>(&num_cols), sizeof(num_cols));

    // Convert from big-endian to host endian
    magic = swap32(magic);
    num_items = swap32(num_items);
    num_rows = swap32(num_rows);
    num_cols = swap32(num_cols);

    // Verify image file format
    if (magic != 0x803) {
        std::cerr << "Invalid image file format" << std::endl;
        return false;
    }

    // Read label file header
    uint32_t label_magic, num_labels;
    label_file.read(reinterpret_cast<char*>(&label_magic), sizeof(label_magic));
    label_file.read(reinterpret_cast<char*>(&num_labels), sizeof(num_labels));

    // Convert label file header
    label_magic = swap32(label_magic);
    num_labels = swap32(num_labels);

    // Verify label file format
    if (label_magic != 0x801) {
        std::cerr << "Invalid label file format" << std::endl;
        return false;
    }

    // Check consistency between images and labels
    if (num_items != num_labels) {
        std::cerr << "Number of images doesn't match number of labels" << std::endl;
        return false;
    }

    // Prepare storage
    samples.resize(num_items);
    std::vector<unsigned char> pixels(num_rows * num_cols);

    // Read and process each sample
    for (uint32_t i = 0; i < num_items; ++i) {
        // Read label
        unsigned char label;
        label_file.read(reinterpret_cast<char*>(&label), 1);
        samples[i].label = static_cast<int>(label);

        // Read image
        image_file.read(reinterpret_cast<char*>(pixels.data()), pixels.size());

        // Normalize pixel values to [0,1]
        for (size_t j = 0; j < pixels.size(); ++j) {
            samples[i].image[j] = static_cast<float>(pixels[j]) / 255.0f;
        }

        // Show progress
        if (i % 1000 == 0) {
            std::cout << "\rLoading data: " << (i * 100.0f / num_items) << "%" << std::flush;
        }
    }
    std::cout << "\rLoading data: 100%" << std::endl;

    return true;
}

int main() {
    auto start_time = std::chrono::high_resolution_clock::now();
    // Declare containers for samples
    std::vector<TrainingSample> train_samples;
    std::vector<TrainingSample> test_samples;

    // Load training data
    if (!loadMNISTImages("./data/MNIST/raw/train-images-idx3-ubyte",
                        "./data/MNIST/raw/train-labels-idx1-ubyte",
                        train_samples)) {
        return -1;
    }
    std::cout << "Successfully loaded " << train_samples.size() << " training samples." << std::endl;

    if (!loadMNISTImages("./data/MNIST/raw/t10k-images-idx3-ubyte",
                        "./data/MNIST/raw/t10k-labels-idx1-ubyte",
                        test_samples)) {
        return -1;
    }
    std::cout << "Successfully loaded " << test_samples.size() << " testing samples." << std::endl;

    int num_trainsamples = train_samples.size();
    int num_testsamples = test_samples.size();

    // Allocate host memory for training data
    float* h_train_images = new float[num_trainsamples * IMAGESIZE];
    int* h_train_labels = new int[num_trainsamples];

    for (int i = 0; i < num_trainsamples; ++i) {
        h_train_labels[i] = train_samples[i].label;
        std::memcpy(&h_train_images[i * IMAGESIZE], train_samples[i].image, sizeof(float) * IMAGESIZE);
    }

    // Allocate GPU memory
    float* d_train_images;
    int* d_train_labels;
    cudaMalloc(&d_train_images, num_trainsamples * IMAGESIZE * sizeof(float));
    cudaMalloc(&d_train_labels, num_trainsamples * sizeof(int));

    // Copy training data to GPU
    cudaMemcpy(d_train_images, h_train_images, num_trainsamples * IMAGESIZE * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_train_labels, h_train_labels, num_trainsamples * sizeof(int), cudaMemcpyHostToDevice);

    // KNN parameters
    int k = 10;  // Number of neighbors
    int correct_predictions = 0;

    // Process each test sample
    for (int t = 0; t < num_testsamples; ++t) {
        // Show progress
        if (t % 100 == 0) {
            std::cout << "\rProcessing test samples: " << (t * 100.0f / num_testsamples) << "%" << std::flush;
        }

        int test_label = test_samples[t].label;

        // Allocate and copy test image to GPU
        float* d_test_image;
        cudaMalloc(&d_test_image, IMAGESIZE * sizeof(float));
        cudaMemcpy(d_test_image, test_samples[t].image, IMAGESIZE * sizeof(float), cudaMemcpyHostToDevice);

        // Configure kernel parameters
        int threadsPerBlock = 256;
        int blocksPerGrid = (num_trainsamples + threadsPerBlock - 1) / threadsPerBlock;

        // Calculate the next power of two for padding
        int pow2_size = 1;
        while (pow2_size < num_trainsamples) pow2_size <<= 1;
        int padded_size = pow2_size;

        // Allocate memory for distances and labels with padding
        float* d_distances;
        int* d_sort_labels;
        cudaMalloc(&d_distances, padded_size * sizeof(float));
        cudaMalloc(&d_sort_labels, padded_size * sizeof(int));

        // Initialize distances and labels with maximum values beyond num_trainsamples
        if (padded_size > num_trainsamples) {
            float max_distance = FLT_MAX;
            int max_label = -1; // Invalid label

            // Create temporary arrays for padding
            float* h_pad_distances = new float[padded_size - num_trainsamples];
            int* h_pad_labels = new int[padded_size - num_trainsamples];
            for (int i = 0; i < padded_size - num_trainsamples; ++i) {
                h_pad_distances[i] = max_distance;
                h_pad_labels[i] = max_label;
            }

            // Copy padding data to device
            cudaMemcpy(d_distances + num_trainsamples, h_pad_distances, (padded_size - num_trainsamples) * sizeof(float), cudaMemcpyHostToDevice);
            cudaMemcpy(d_sort_labels + num_trainsamples, h_pad_labels, (padded_size - num_trainsamples) * sizeof(int), cudaMemcpyHostToDevice);

            // Free temporary host arrays
            delete[] h_pad_distances;
            delete[] h_pad_labels;
        }

        // The rest of your code remains mostly the same
        // Compute distances
        computeEuclideanDistances<<<blocksPerGrid, threadsPerBlock>>>(
            d_train_images, d_test_image, d_distances, d_sort_labels, d_train_labels, num_trainsamples
        );

        // Call the updated bitonicSort function
        bitonicSort(d_distances, d_sort_labels, num_trainsamples);

        // When copying back the top k elements, ensure you only consider the valid data
        float* h_distances = new float[k];
        int* h_labels = new int[k];
        cudaMemcpy(h_distances, d_distances, k * sizeof(float), cudaMemcpyDeviceToHost);
        cudaMemcpy(h_labels, d_sort_labels, k * sizeof(int), cudaMemcpyDeviceToHost);

        // Count label frequencies
        std::vector<int> labelCount(10, 0);
        for (int i = 0; i < k; ++i) {
            labelCount[h_labels[i]]++;
        }

        // Find most common label
        int predictedLabel = std::distance(labelCount.begin(),
                                         std::max_element(labelCount.begin(), labelCount.end()));

        // Update accuracy counter
        if (predictedLabel == test_label) {
            correct_predictions++;
        }

        // Print periodic updates
        if (t % 1000 == 0) {
            std::cout << "\nTest Sample " << t << ": Actual = " << test_label
                     << ", Predicted = " << predictedLabel << std::endl;
        }

        // Free temporary memory
        delete[] h_distances;
        delete[] h_labels;
        cudaFree(d_distances);
        cudaFree(d_sort_labels);
        cudaFree(d_test_image);
    }

    // Print final progress
    std::cout << "\rProcessing test samples: 100%" << std::endl;

    // Calculate and display final results
    float accuracy = (float)correct_predictions / num_testsamples * 100.0f;
    std::cout << "\nFinal Results:" << std::endl;
    std::cout << "Total test samples: " << num_testsamples << std::endl;
    std::cout << "Correct predictions: " << correct_predictions << std::endl;
    std::cout << "Accuracy: " << accuracy << "%" << std::endl;

    auto end_time = std::chrono::high_resolution_clock::now();
    auto duration = std::chrono::duration_cast<std::chrono::milliseconds>(end_time - start_time);
    std::cout << "Total execution time: " << duration.count() / 1000.0 << " seconds" << std::endl;

    // Free all allocated memory
    delete[] h_train_images;
    delete[] h_train_labels;
    cudaFree(d_train_images);
    cudaFree(d_train_labels);

    return 0;
}

In [ ]:
%cuda_group_run --group "knn" --compiler-args "-O3 -g -std=c++20 -arch=sm_75"

Loading data: 100%333%
Successfully loaded 60000 training samples.
Loading data: 100%
Successfully loaded 10000 testing samples.
Processing test samples: 0%
Test Sample 0: Actual = 7, Predicted = 7
Processing test samples: 10%
Test Sample 1000: Actual = 9, Predicted = 9
Processing test samples: 20%
Test Sample 2000: Actual = 6, Predicted = 6
Processing test samples: 30%
Test Sample 3000: Actual = 6, Predicted = 6
Processing test samples: 40%
Test Sample 4000: Actual = 9, Predicted = 4
Processing test samples: 50%
Test Sample 5000: Actual = 3, Predicted = 3
Processing test samples: 60%
Test Sample 6000: Actual = 9, Predicted = 9
Processing test samples: 70%
Test Sample 7000: Actual = 1, Predicted = 1
Processing test samples: 80%
Test Sample 8000: Actual = 4, Predicted = 4
Processing test samples: 90%
Test Sample 9000: Actual = 7, Predicted = 7
Processing test samples: 100%

Final Results:
Total test samples: 10000
Correct predictions: 9665
Accuracy: 96.65%

